# Syft Principles
- Everything is a Tensor
- All execution happens on a Worker via a Client
- Security is at the Tensor Level
- Everything can be packaged as a Plan or Protocol
- All objects are serializable via Protobuf to Polyglot Runtimes

# PySyft Principles
- Enforce only kwargs when possible
- Autoamtic type checking (using type hints and @check.type_hints decorator) for all func/method arguments except self, *args, and **kwargs 
- Inline document everything (inline documentation should be good enough to be the only resource someone needs to learn the codebase)
- Minimize framework specific code
- Code generate where possible (both functionality and tests)

# Dev Principles
- Commit messages worded as commands
- Always push to a WIP PR

# PySyft Custom Tensor Principles
- No functionality is ever duplicated across tensor types
- All custom tensor types are subclassed so indicate what tensor types they can be children of (and what subclasses can be their children). Please avoid subclassing from AbstractTensor or RestrictedTensor directly.

# New Features We Want
- PyTorch has new subclassing capabilities
- Serialization should all use Protobuf
- Multi-framework support. 
- Code generate framework specific tensors based on a SyftTensor abstraction
- Automatic testing for all permuations of tensor abstractions, across all methods, and all frameworks
- All tensor abstractions should have a type inhereitance which limits their use

### Steps Per Framework:

1. Setup Global Flags
    - syft.typecheck (bool): specifies whether to actively typecheck or not

2. Custom Constructor method
    - All Frameworks / All Tensor Types
        - pre_init(*args, **kwargs)
        - init()
        - post_init(*args, **kwargs)
    
3. Reliably generate unique tensor ID (tensor.id)
    - All Tensorflow Tensor Types:  
        - use internal experimental id attribute
        - during post_init, if ID is non-unique, call .regeenerate_id() which 
            re-initialize the tensor again (with the same children) until you get a unique one
    - All Torch/Numpy Tensor Types:
        - use random number, store on tensor (only attribute to be stored this way)
        - during post_init, if ID is non-unique, call .regenerate_id() which will recreate a new one

3.1 Generic Types (list, tuple, dict, string, int, set, boolean)

4. BaseWorker
    - Object Store
    - set_obj()
    - get_obj()

    - id_exists functionality
    
5. Reliably Register and Delete Objects
    - All Frameworks / All Tensor Types / All Objects:
        - post_init() should register the tensor with the correct BaseWorker object. 
        - del my_tensor should de-register the tensor

6. Custom attributes via attribute store (id -> attributes [key -> value])
    - All Frameworks / All Tensor Types
        - __setattr__(attribute=value) 
        - __getattr__("attribute") and 
            - overload __hasattr__ too
    
7. Custom Tensor Type
    - All Frameworks / All Tensor Types
    - Should be able to subclass and extend custom tensor types
    - Initial tensor types should include:
        - RestrictedTensor: all methods but no functionality
        - AbstractTensor: all methods with default torch functionality (i.e., just pass through to .child)
        - "PlusIsMinusTensor" which executes subtraction when addition is called

8. Syft Tensor Type -> CodeGen Custom Tensor Types
    - Use AST to code generate all custom tensors from a base SyftTensor type

9. Automatic Testing Suite
    - Automatically test a custom tensor with all tensor methods in all frameworks. Every operation should either produce the same results (within an error bound) as the original version of a PyTorch tensor or it should raise a NotImplemented exception (explicitly) with an error message saying that this operation is not supporteed by a specific custom tensor type.
    - Note:
        - Ideally we would just keep one master list of methods for each framework
        - We would add a tensor type to a list of tensor types to be tested, possibly with a method for how to instantiate the tensor and how to convert its values to something comparable with each respective original tensor

9b. Automatic Documentation?

10. Custom Tensor Function
    - 

11. Automatic Testing Suite Improvements
    
    - Automatically test all custom tensor types with all tensor functions in all frameworks. Every operation should either produce the same results (within an error bound) as the original version of a PyTorch tensor or it should raise a NotImplemented exception (explicitly) with an error message saying that this operation is not supporteed by a specific custom tensor type.

12. Tensor Chaining
    - Include type checking for tensor types

13. Automatic Testing Suite Improvements

    - We should be able to automatically permute over every plausible chain of tensor abstractions and test them for all methods and functions

14. original Tracing (JIT), Plan, MapPlan, ReducePlan

15. Automatic Testing Suite Improvements

    - For all methods and functions on all tensor chains in all frameworks we should be able to test that JIT works correctly, building plans and executing them.
    
16. AutogradTensor
    - We should be able to specify an arbitrary point in the chain to run autograd by inserting an AutogradTensor at that point.
    
17. Automatic Testing Suite Improvements

    - For all methods and functions on all tensor chains in all frameworks both JIT traced in plans and executed eagerly, we should be able to test that backpropagation works correctly at any (continuous/floating point) level in the tensor chain hierarchy.
    
18. Karl's Worker, Operation, and Command Abstractions
    
19. PointerTensor

20. Automatic Testing Suite Improvements

    - For all methods and functions on all tensor chains in all frameworks both JIT traced in plans and executed eagerly, we should be able to test that backpropagation works correctly at any (continuous/floating point) level in the tensor chain hierarchy. This includes PointerTensors

21. PromiseTensor

22. Automatic Testing Suite Improvements

    - For all methods and functions on all tensor chains in all frameworks both JIT traced in plans and executed eagerly, we should be able to test that backpropagation works correctly at any (continuous/floating point) level in the tensor chain hierarchy. This includes PromiseTensors

23. Protocol

24. Automatic Testing Suite Improvements

    - For all methods and functions on all tensor chains in all frameworks both JIT traced in plans and executed eagerly, we should be able to test that backpropagation works correctly at any (continuous/floating point) level in the tensor chain hierarchy. This includes Protocols
    
25. ThreadWorker

